# APLICACIÓN DE MODELOS DE APRENDIZAJE AUTOMÁTICO PARA LA PREDICCIÓN DEL PRECIO DEL PETRÓLEO BRENT

### Autor: Oscar Aliaga Gonzales

## Extraccion de datos a la API Yahoo Finance 

In [1]:

import yfinance as yf
import pandas as pd

# Definir el ticker del Petróleo Brent
ticker = "BZ=F"

# Crear un objeto Ticker
brent = yf.Ticker(ticker)

# Definir las fechas de inicio y fin
start_date = "2007-07-30"
end_date = "2024-04-16"

# Extraer los datos históricos con las fechas específicas
data_brent = brent.history(start=start_date, end=end_date)

# Mostrar los primeros y últimos registros para verificar
print(data_brent.head())
print(data_brent.tail())

                                Open       High        Low      Close  Volume  \
Date                                                                            
2007-07-30 00:00:00-04:00  75.849998  76.529999  75.440002  75.739998    2575   
2007-07-31 00:00:00-04:00  75.699997  77.169998  75.669998  77.050003    3513   
2007-08-01 00:00:00-04:00  77.000000  77.059998  74.860001  75.349998    3930   
2007-08-02 00:00:00-04:00  75.220001  76.209999  74.269997  75.760002    6180   
2007-08-03 00:00:00-04:00  75.389999  76.000000  74.529999  74.750000    4387   

                           Dividends  Stock Splits  
Date                                                
2007-07-30 00:00:00-04:00        0.0           0.0  
2007-07-31 00:00:00-04:00        0.0           0.0  
2007-08-01 00:00:00-04:00        0.0           0.0  
2007-08-02 00:00:00-04:00        0.0           0.0  
2007-08-03 00:00:00-04:00        0.0           0.0  
                                Open       High        Low   

In [2]:
# Eliminar filas con valores faltantes
data_brent.dropna(inplace=True)

# Verificar tipos de datos y realizar conversiones si es necesario
print(data_brent.dtypes)

# Si necesitas resetear el índice para que la fecha sea una columna
data_brent.reset_index(inplace=True)

# Mostrar el DataFrame limpio
print(data_brent.head())

Open            float64
High            float64
Low             float64
Close           float64
Volume            int64
Dividends       float64
Stock Splits    float64
dtype: object
                       Date       Open       High        Low      Close  \
0 2007-07-30 00:00:00-04:00  75.849998  76.529999  75.440002  75.739998   
1 2007-07-31 00:00:00-04:00  75.699997  77.169998  75.669998  77.050003   
2 2007-08-01 00:00:00-04:00  77.000000  77.059998  74.860001  75.349998   
3 2007-08-02 00:00:00-04:00  75.220001  76.209999  74.269997  75.760002   
4 2007-08-03 00:00:00-04:00  75.389999  76.000000  74.529999  74.750000   

   Volume  Dividends  Stock Splits  
0    2575        0.0           0.0  
1    3513        0.0           0.0  
2    3930        0.0           0.0  
3    6180        0.0           0.0  
4    4387        0.0           0.0  


In [3]:
# Guardar los datos en un archivo CSV para futuros análisis
data_brent.to_csv('brent_oil_prices.csv', index=False)

## Analisis exploratorio de los datos

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Cargar los datos desde el archivo CSV
data_brent = pd.read_csv('brent_oil_prices.csv')

# Convertir la columna 'Date' a tipo datetime si aún no lo está
data_brent['Date'] = pd.to_datetime(data_brent['Date'])

# Establecer la columna 'Date' como índice del DataFrame
data_brent.set_index('Date', inplace=True)

# Crear la gráfica
fig = go.Figure()

# Añadir la línea del precio de cierre
fig.add_trace(go.Scatter(x=data_brent.index, y=data_brent['Close'], mode='lines', name='Precio de Cierre', line=dict(color='blue')))

# Configurar el título y los ejes
fig.update_layout(
    title='Evolución Temporal del Precio de Cierre del Petróleo Brent',
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    legend=dict(x=0, y=1),
    hovermode='x unified',
    template='plotly_white'
)

# Mostrar la gráfica en Jupyter Notebook
fig.show()


![Evolución Temporal del Precio de Cierre del Petróleo Brent](Graficas%20de%20resultados/Evoluci%C3%B3n%20Temporal%20del%20Precio%20de%20Cierre%20del%20Petr%C3%B3leo%20Brent.png)

In [4]:
import pandas as pd
import plotly.graph_objects as go

# Cargar los datos desde el archivo CSV
data_brent = pd.read_csv('brent_oil_prices.csv')

# Convertir la columna 'Date' a tipo datetime si aún no lo está
data_brent['Date'] = pd.to_datetime(data_brent['Date'])

# Establecer la columna 'Date' como índice del DataFrame
data_brent.set_index('Date', inplace=True)

# Crear la gráfica
fig = go.Figure()

# Añadir la línea del precio de cierre
fig.add_trace(go.Scatter(x=data_brent.index, y=data_brent['Close'], mode='lines', name='Precio de Cierre', line=dict(color='blue')))

# Configurar el título y los ejes
fig.update_layout(
    title={
        'text': 'Evolución Temporal del Precio de Cierre del Petróleo Brent',
        'x': 0.5,  # Centrando el título
        'xanchor': 'center'
    },
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    legend=dict(x=0, y=1),
    hovermode='x unified',
    template='plotly_white'
)

# Mostrar la gráfica en Jupyter Notebook
fig.show()


## Random Forest

### Predicciones Diarias

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import time  # Importar la librería time para medir el tiempo

# Cargar los datos
data_brent = pd.read_csv('brent_oil_prices.csv')
data_brent['Date'] = pd.to_datetime(data_brent['Date'], errors='coerce', utc=True)
data_brent.set_index('Date', inplace=True)

# Preparar los datos para Random Forest
scaler = MinMaxScaler(feature_range=(0, 1))
data_brent_scaled = scaler.fit_transform(data_brent[['Close']])

# Crear secuencias
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(seq_length, len(data)):
        xs.append(data[i-seq_length:i].flatten())
        ys.append(data[i][0])
    return np.array(xs), np.array(ys)

seq_length = 100
x, y = create_sequences(data_brent_scaled, seq_length)

# División de datos en entrenamiento y prueba
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Instanciar el modelo Random Forest con los mejores parámetros encontrados
rf_model = RandomForestRegressor(bootstrap=True, max_depth=20, max_features='sqrt', 
                                 min_samples_leaf=4, min_samples_split=10, n_estimators=100, 
                                 random_state=42)

# Iniciar el contador de tiempo antes de entrenar el modelo
start_time = time.time()

# Entrenar el modelo
rf_model.fit(x_train, y_train)

# Detener el contador de tiempo después de entrenar el modelo y calcular la duración
end_time = time.time()
training_time = end_time - start_time

# Realizar predicciones con el conjunto de prueba
y_pred = rf_model.predict(x_test)

# Calcular las métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")
print(f"Tiempo de entrenamiento: {training_time} segundos")  # Mostrar el tiempo de entrenamiento

# Desnormalizar las predicciones y los valores de prueba
y_pred_real = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1))

# Crear un DataFrame para facilitar el trazado
df_result = pd.DataFrame({'real': y_test_real.flatten(), 'pred': y_pred_real.flatten()}, index=data_brent.index[-len(y_test_real):])

# Gráfico de comparación utilizando Plotly
fig = go.Figure()

# Añadir las series reales y predichas al gráfico
fig.add_trace(go.Scatter(x=df_result.index, y=df_result['real'], mode='lines', name='Precio Real'))
fig.add_trace(go.Scatter(x=df_result.index, y=df_result['pred'], mode='lines', name='Predicciones Random Forest'))

# Configurar el título y los ejes
fig.update_layout(
        title={
        'text': 'Comparación entre Precios Reales y Predicciones diarias utilizando Random Forest',
        'x': 0.5,  # Centrando el título
        'xanchor': 'center'
    },
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    legend=dict(x=0, y=1),
    hovermode='x unified',
    template='plotly_white'
)

# Mostrar la gráfica
fig.show()


MAE: 0.018865573701435264
MSE: 0.0006814622189725003
RMSE: 0.0261048313339217
R^2: 0.9389932468664858
Tiempo de entrenamiento: 2.2680046558380127 segundos


![Comparación entre Precios Reales y Predicciones diarias utilizando Random Forest](Graficas%20de%20resultados/Comparaci%C3%B3n%20entre%20Precios%20Reales%20y%20Predicciones%20diarias%20utilizando%20Random%20Forest.png)

In [23]:
from sklearn.model_selection import learning_curve

# Cálculo de las curvas de aprendizaje
train_sizes, train_scores, test_scores = learning_curve(rf_model, x, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, train_sizes=np.linspace(0.01, 1.0, 50))

# Calcular la media y la desviación estándar del error de entrenamiento y prueba
train_mean = -np.mean(train_scores, axis=1)
test_mean = -np.mean(test_scores, axis=1)

# Gráfico de la curva de aprendizaje
fig_learning_curve = go.Figure()

# Curva de entrenamiento
fig_learning_curve.add_trace(go.Scatter(x=train_sizes, y=train_mean, mode='lines', name='Pérdida de Entrenamiento', line=dict(color='blue')))

# Curva de validación
fig_learning_curve.add_trace(go.Scatter(x=train_sizes, y=test_mean, mode='lines', name='Pérdida de Validación', line=dict(color='red')))

# Configurar el título y los ejes
fig_learning_curve.update_layout(
    title={
        'text': 'Curva de Aprendizaje de la prediccion diaria de Random Forest',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Tamaño del Conjunto de Entrenamiento',
    yaxis_title='MSE Negativo',
    legend=dict(x=0.5, y=1.0, xanchor='center'),
    hovermode='x unified'
)

# Mostrar la gráfica
fig_learning_curve.show()

![Curvas de Aprendizaje de la prediccion diaria de Random Forest](Graficas%20de%20resultados/Curvas%20de%20Aprendizaje%20de%20la%20prediccion%20diaria%20de%20Random%20Forest.png)

In [9]:


import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import plotly.graph_objects as go

# Cargar los datos
data_brent = pd.read_csv('brent_oil_prices.csv')
data_brent['Date'] = pd.to_datetime(data_brent['Date'], errors='coerce', utc=True)
data_brent.set_index('Date', inplace=True)

# Preparar los datos para Random Forest
scaler = MinMaxScaler(feature_range=(0, 1))
data_brent_scaled = scaler.fit_transform(data_brent[['Close']])

# Crear secuencias
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(seq_length, len(data)):
        xs.append(data[i-seq_length:i].flatten())
        ys.append(data[i][0])
    return np.array(xs), np.array(ys)

seq_length = 100
x, y = create_sequences(data_brent_scaled, seq_length)

# División de datos en entrenamiento y prueba
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Instanciar el modelo Random Forest con los mejores parámetros encontrados
rf_model = RandomForestRegressor(bootstrap=True, max_depth=20, max_features='sqrt', 
                                 min_samples_leaf=4, min_samples_split=10, n_estimators=100, 
                                 random_state=42)

# Entrenar el modelo
rf_model.fit(x_train, y_train)

# Realizar predicciones con el conjunto de prueba
y_pred = rf_model.predict(x_test)

# Calcular las métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

# Desnormalizar las predicciones y los valores de prueba
y_pred_real = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1))

# Crear un DataFrame para facilitar el trazado
df_result = pd.DataFrame({'real': y_test_real.flatten(), 'pred': y_pred_real.flatten()}, index=data_brent.index[-len(y_test_real):])

# Gráfico de comparación utilizando Plotly
fig = go.Figure()

# Añadir las series reales y predichas al gráfico
fig.add_trace(go.Scatter(x=df_result.index, y=df_result['real'], mode='lines', name='Precio Real'))
fig.add_trace(go.Scatter(x=df_result.index, y=df_result['pred'], mode='lines', name='Predicciones Random Forest'))

# Configurar el título y los ejes
fig.update_layout(
        title={
        'text': 'Comparación entre Precios Reales y Predicciones utilizando Random Forest',
        'x': 0.5,  # Centrando el título
        'xanchor': 'center'
    },
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    legend=dict(x=0, y=1),
    hovermode='x unified',
    template='plotly_white'
)

# Mostrar la gráfica
fig.show()

# Curvas de Aprendizaje
train_sizes = np.linspace(0.1, 1.0, 10)
train_scores = []
val_scores = []

for train_size in train_sizes:
    x_train_size = x_train[:int(len(x_train) * train_size)]
    y_train_size = y_train[:int(len(y_train) * train_size)]
    rf_model.fit(x_train_size, y_train_size)
    y_train_pred = rf_model.predict(x_train_size)
    y_val_pred = rf_model.predict(x_test)
    train_scores.append(mean_squared_error(y_train_size, y_train_pred))
    val_scores.append(mean_squared_error(y_test, y_val_pred))

# Gráfico de Curvas de Aprendizaje utilizando Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=train_sizes, y=train_scores, mode='lines', name='Pérdida de Entrenamiento'))
fig.add_trace(go.Scatter(x=train_sizes, y=val_scores, mode='lines', name='Pérdida de Validación'))

fig.update_layout(
    title='Curvas de Aprendizaje de Random Forest',
    xaxis_title='Proporción del Conjunto de Entrenamiento',
    yaxis_title='Error Cuadrático Medio',
    legend=dict(x=0, y=1),
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

# Validación Cruzada con Series Temporales
tscv = TimeSeriesSplit(n_splits=5)
cv_scores = cross_val_score(rf_model, x, y, cv=tscv, scoring='neg_mean_squared_error')
print("Errores CV (MSE) para cada partición:", -cv_scores)
print("Error CV (MSE) promedio:", -cv_scores.mean())
print("Error CV (MSE) std:", -cv_scores.std())


MAE: 0.018865573701435264
MSE: 0.0006814622189725003
RMSE: 0.0261048313339217
R^2: 0.9389932468664858


Errores CV (MSE) para cada partición: [0.00096606 0.00261397 0.00038456 0.00062294 0.00078127]
Error CV (MSE) promedio: 0.001073759471312183
Error CV (MSE) std: -0.0007934268160005713


### predicciones semanales

In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go
import time 

# Cargar los datos asegurando que la columna 'Date' se analice como fecha
data_brent = pd.read_csv('brent_oil_prices.csv', parse_dates=['Date'])
data_brent.set_index('Date', inplace=True)

# Convertir el índice a DatetimeIndex asegurando que se manejen las zonas horarias
if not isinstance(data_brent.index, pd.DatetimeIndex):
    data_brent.index = pd.to_datetime(data_brent.index, utc=True)  # Uso de utc=True para conversiones tz-aware

# Agrupar datos por semana y obtener el precio de cierre promedio de la semana
data_brent_weekly = data_brent['Close'].resample('W').mean()

# Manejo de valores NaN - relleno hacia adelante
data_brent_weekly.fillna(method='ffill', inplace=True)  # Puedes cambiar a 'bfill' si prefieres rellenar hacia atrás

# Preparar los datos para Random Forest
scaler = MinMaxScaler(feature_range=(0, 1))
data_brent_scaled = scaler.fit_transform(data_brent_weekly.values.reshape(-1, 1))

# Crear secuencias
def create_features(data, seq_length):
    x, y = [], []
    for i in range(seq_length, len(data)):
        x.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(x), np.array(y)

seq_length = 10  # Ajuste de la longitud de secuencia para datos semanales
x, y = create_features(data_brent_scaled, seq_length)

# División de datos en entrenamiento y prueba
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Modelo Random Forest
# Instanciar el modelo Random Forest con los mejores parámetros encontrados
model_rf = RandomForestRegressor(bootstrap=False, max_depth=None, max_features='log2', 
                                 min_samples_leaf=2, min_samples_split=2, n_estimators=200, 
                                 random_state=42)
start_time = time.time()  # Iniciar el contador de tiempo
model_rf.fit(x_train, y_train)
end_time = time.time()  # Detener el contador de tiempo

# Calcular y mostrar la duración del entrenamiento
training_duration = end_time - start_time
print(f"Tiempo de entrenamiento: {training_duration:.3f} segundos")

# Predicciones y evaluación
y_pred = model_rf.predict(x_test)
# Des-normalizar las predicciones y los valores reales para el cálculo de las métricas
y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_real = scaler.inverse_transform(y_pred.reshape(-1, 1))

# Calcular las métricas
mse_rf = mean_squared_error(y_test_real, y_pred_real)
mae_rf = mean_absolute_error(y_test_real, y_pred_real)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test_real, y_pred_real)

print("Error Cuadrático Medio (MSE) de Random Forest:", mse_rf)
print("Error Absoluto Medio (MAE) de Random Forest:", mae_rf)
print("Raíz del Error Cuadrático Medio (RMSE) de Random Forest:", rmse_rf)
print("Coeficiente de Determinación (R^2) de Random Forest:", r2_rf)

# Gráfico de comparación utilizando Plotly
fig = go.Figure()

# Añadir las series reales y predichas al gráfico
pred_index = data_brent_weekly.index[-len(y_pred):]
fig.add_trace(go.Scatter(x=pred_index, y=y_test_real.flatten(), mode='lines', name='Precio Real'))
fig.add_trace(go.Scatter(x=pred_index, y=y_pred_real.flatten(), mode='lines', name='Predicciones Random Forest'))

# Configurar el título y los ejes
fig.update_layout(
    title={
        'text': 'Comparación de Precios Reales y Predicciones Semanales con Random Forest',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    legend=dict(x=0, y=1),
    hovermode='x unified',
    template='plotly_white'
)

# Mostrar la gráfica
fig.show()


Tiempo de entrenamiento: 0.530 segundos
Error Cuadrático Medio (MSE) de Random Forest: 17.366469021141906
Error Absoluto Medio (MAE) de Random Forest: 3.10435114314515
Raíz del Error Cuadrático Medio (RMSE) de Random Forest: 4.167309566271974
Coeficiente de Determinación (R^2) de Random Forest: 0.9103609239189413


![Comparación de Precios Reales y Predicciones Semanales con Random Forest](Graficas%20de%20resultados/Comparaci%C3%B3n%20de%20Precios%20Reales%20y%20Predicciones%20Semanales%20con%20Random%20Forest.png)

In [27]:
import plotly.graph_objects as go
from sklearn.model_selection import learning_curve

# Definir la función para graficar las curvas de aprendizaje con Plotly
def plot_learning_curves_plotly(model, X, y):
    train_sizes, train_scores, test_scores = learning_curve(model, X, y,
                                                            train_sizes=np.linspace(0.1, 1.0, 10),
                                                            cv=5, scoring='neg_mean_squared_error',
                                                            n_jobs=-1)
    # Calcular la media de los puntajes de entrenamiento y prueba
    train_mean = -np.mean(train_scores, axis=1)
    test_mean = -np.mean(test_scores, axis=1)

    # Crear figura
    fig = go.Figure()

    # Añadir trazas para los puntajes de entrenamiento y validación
    fig.add_trace(go.Scatter(x=train_sizes, y=train_mean,
                             mode='lines',
                             name='Pérdida entrenamiento',
                             line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=train_sizes, y=test_mean,
                             mode='lines',
                             name='Pérdida validación',
                             line=dict(color='red')))

    # Actualizar el diseño del gráfico
    fig.update_layout(
                          title={
                            'text': 'Curvas de aprendizaje de la prediccion semanal de Random Forest',
                            'y':0.9,
                            'x':0.5,
                            'xanchor': 'center',
                            'yanchor': 'top'
    },
                      xaxis_title='Tamaño del conjunto de entrenamiento',
                      yaxis_title='MSE',
    legend=dict(x=0.5, y=1.0, xanchor='center'),
    hovermode='x unified')

    # Mostrar figura
    fig.show()

# Llamar a la función con el modelo y los datos
plot_learning_curves_plotly(model_rf, x, y)



![Curvas de aprendizaje de la prediccion semanal de Random Forest](Graficas%20de%20resultados/Curvas%20de%20aprendizaje%20de%20la%20prediccion%20semanal%20de%20Random%20Forest.png)

## GBR

### predicciones diarias

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import time
import plotly.graph_objects as go

# Cargar los datos
data_brent = pd.read_csv('brent_oil_prices.csv')
data_brent['Date'] = pd.to_datetime(data_brent['Date'])
data_brent.set_index('Date', inplace=True)

# Preparar los datos para Gradient Boosting Machine
scaler = MinMaxScaler(feature_range=(0, 1))
data_brent_scaled = scaler.fit_transform(data_brent[['Close']])

# Crear secuencias
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(seq_length, len(data)):
        xs.append(data[i-seq_length:i].flatten())
        ys.append(data[i][0])
    return np.array(xs), np.array(ys)

seq_length = 100
x, y = create_sequences(data_brent_scaled, seq_length)

# División de datos en entrenamiento y prueba
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Instanciar el modelo Gradient Boosting Machine con parámetros 
gbm_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, min_samples_split=10, 
                                      min_samples_leaf=4, max_features='sqrt', 
                                      learning_rate=0.1, random_state=42)

# Medir el tiempo de entrenamiento
start_time = time.time()
gbm_model.fit(x_train, y_train)
end_time = time.time()
training_time = end_time - start_time

# Realizar predicciones con el conjunto de prueba
y_pred = gbm_model.predict(x_test)

# Calcular las métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Tiempo de entrenamiento: {training_time:.3f} segundos")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

# Desnormalizar las predicciones y los valores reales
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1))

# Crear un DataFrame para facilitar el trazado
df = pd.DataFrame({'Precio real': y_test_inv.flatten(), 'Precio predicciones': y_pred_inv.flatten()})

# Trazar los valores reales y predichos con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(df)), y=df['Precio real'], mode='lines', name='Real'))
fig.add_trace(go.Scatter(x=np.arange(len(df)), y=df['Precio predicciones'], mode='lines', name='Predicho'))
fig.update_layout(title={'text': "Comparación de Precios Reales y Predicciones diarias utilizando GBR", 'y':0.9,'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  xaxis_title='Numero de observaciones',
                  yaxis_title='Precio',
                  legend=dict(x=0, y=1),
                  hovermode='x unified',
                  template='plotly_white')
fig.show()

Tiempo de entrenamiento: 1.998 segundos
MAE: 0.019501060114936335
MSE: 0.0006973592803478244
RMSE: 0.026407561045045876
R^2: 0.9375700893210315


![Comparación de Precios Reales y Predicciones diarias utilizando GBR](Graficas%20de%20resultados/Comparaci%C3%B3n%20de%20Precios%20Reales%20y%20Predicciones%20diarias%20utilizando%20GBR.png)

In [38]:
from sklearn.model_selection import learning_curve

# Función para graficar las curvas de aprendizaje ajustada
def plot_learning_curves(model, X, y):
    train_sizes, train_scores, test_scores = learning_curve(
        model, X, y, cv=5, scoring='neg_mean_squared_error',
        train_sizes=np.linspace(0.1, 1.0, 10), n_jobs=-1)

    train_mean = -np.mean(train_scores, axis=1)
    test_mean = -np.mean(test_scores, axis=1)

    fig = go.Figure()

    # Curva de entrenamiento ajustada
    fig.add_trace(go.Scatter(x=train_sizes, y=train_mean, mode='lines',
                             name='Perdida Entrenamiento', line=dict(color='blue')))

    # Curva de validación cruzada ajustada
    fig.add_trace(go.Scatter(x=train_sizes, y=test_mean, mode='lines',
                             name='Perdida Validación', line=dict(color='red')))

    fig.update_layout(title={'text': "Curvas de aprendizaje de prediccion diaria para Gradient Boosting Regressor", 'x': 0.5},
                      xaxis_title='Tamaño del conjunto de entrenamiento',
                      yaxis_title='Negativo del MSE',
                      template='plotly_white')

    fig.show()

# Llamar a la función ajustada para graficar las curvas de aprendizaje
plot_learning_curves(gbm_model, x, y)

![Curvas de aprendizaje de prediccion diaria para GBR](Graficas%20de%20resultados/Curvas%20de%20aprendizaje%20de%20prediccion%20diaria%20para%20GBR.png)

### predicciones semanales

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import time
import plotly.graph_objects as go

# Cargar los datos asegurando que la columna 'Date' se analice como fecha
data_brent = pd.read_csv('brent_oil_prices.csv', parse_dates=['Date'])
data_brent.set_index('Date', inplace=True)

# Convertir el índice a DatetimeIndex asegurando que se manejen las zonas horarias
if not isinstance(data_brent.index, pd.DatetimeIndex):
    data_brent.index = pd.to_datetime(data_brent.index, utc=True)

# Agrupar datos por semana y obtener el precio de cierre promedio de la semana
data_brent_weekly = data_brent['Close'].resample('W').mean()

# Manejo de valores NaN - relleno hacia adelante
data_brent_weekly.fillna(method='ffill', inplace=True)

# Preparar los datos para el modelo
scaler = MinMaxScaler(feature_range=(0, 1))
data_brent_scaled = scaler.fit_transform(data_brent_weekly.values.reshape(-1, 1))

# Crear secuencias
def create_features(data, seq_length):
    x, y = [], []
    for i in range(seq_length, len(data)):
        x.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(x), np.array(y)

seq_length = 20
x, y = create_features(data_brent_scaled, seq_length)

# División de datos en entrenamiento y prueba
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Instanciar el modelo
gbm_model = GradientBoostingRegressor(n_estimators=300, max_depth=3, min_samples_split=5, 
                                      min_samples_leaf=2, max_features='sqrt', learning_rate=0.1,
                                      random_state=42)

# Iniciar el cronómetro para calcular el tiempo de entrenamiento
start_time = time.time()

# Entrenar el modelo
gbm_model.fit(x_train, y_train)

# Detener el cronómetro
end_time = time.time()
training_time = end_time - start_time

# Mostrar el tiempo de entrenamiento
print(f"Tiempo de entrenamiento del modelo: {training_time:.3f} segundos")

# Realizar predicciones
y_pred = gbm_model.predict(x_test)

# Calcular métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

# Desnormalizar las predicciones y los valores reales
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1))

# Crear un DataFrame para facilitar el trazado
df = pd.DataFrame({'Real': y_test_inv.flatten(), 'Predicho': y_pred_inv.flatten()})

# Trazar los valores reales y predichos usando Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(df)), y=df['Real'], mode='lines', name='Real'))
fig.add_trace(go.Scatter(x=np.arange(len(df)), y=df['Predicho'], mode='lines', name='Predicho'))
fig.update_layout(title={'text': "Comparativa de precios reales y precios predichos semanales de GBR", 'x': 0.5},
                  xaxis_title='Observaciones',
                  yaxis_title='Precio',
                  template='plotly_white')
fig.show()

Tiempo de entrenamiento del modelo: 0.434 segundos
MAE: 0.027317365835745084
MSE: 0.0013455592954245707
RMSE: 0.0366818660297506
R^2: 0.8929933314956844


![Comparativa de precios reales y precios predichos semanales de GBR](Graficas%20de%20resultados/Comparativa%20de%20precios%20reales%20y%20precios%20predichos%20semanales%20de%20GBR.png)

In [41]:
from sklearn.model_selection import learning_curve
import plotly.graph_objects as go

def plot_learning_curves(model, X, y):
    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 5))
    
    train_mean = -np.mean(train_scores, axis=1)
    test_mean = -np.mean(test_scores, axis=1)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_sizes, y=train_mean, mode='lines', name='Perdida Entrenamiento'))
    fig.add_trace(go.Scatter(x=train_sizes, y=test_mean, mode='lines', name='Perdida Validación'))
    
    fig.update_layout(title={'text': "Curvas de Aprendizaje de predicciones semanales del GBR", 'x': 0.5},
                      xaxis_title='Tamaño del conjunto de entrenamiento',
                      yaxis_title='Error (MSE)',
                      template='plotly_white')
    fig.show()

# Asegúrate de tener x e y definidos como tus datos de entrada y salida
plot_learning_curves(gbm_model, x, y)

![Curvas de Aprendizaje de predicciones semanales del GBR](Graficas%20de%20resultados/Curvas%20de%20Aprendizaje%20de%20predicciones%20semanales%20del%20GBR.png)

## ADA Boost

### predicciones diarias

In [44]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import time
import plotly.graph_objects as go

# Cargar los datos
data_brent = pd.read_csv('brent_oil_prices.csv')
data_brent['Date'] = pd.to_datetime(data_brent['Date'])
data_brent.set_index('Date', inplace=True)

# Preparar los datos para AdaBoost
scaler = MinMaxScaler(feature_range=(0, 1))
data_brent_scaled = scaler.fit_transform(data_brent[['Close']])

# Crear secuencias
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(seq_length, len(data)):
        xs.append(data[i-seq_length:i].flatten())
        ys.append(data[i][0])
    return np.array(xs), np.array(ys)

seq_length = 100
x, y = create_sequences(data_brent_scaled, seq_length)

# División de datos en entrenamiento y prueba
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Instanciar el modelo AdaBoost con DecisionTreeRegressor como base
base_estimator = DecisionTreeRegressor(max_depth=4, min_samples_leaf=4, random_state=42)
ada_model = AdaBoostRegressor(base_estimator=base_estimator, n_estimators=50, learning_rate=0.01, random_state=42)

# Iniciar el cronómetro para calcular el tiempo de entrenamiento
start_time = time.time()

# Entrenar el modelo
ada_model.fit(x_train, y_train)

# Detener el cronómetro
end_time = time.time()
training_time = end_time - start_time

# Mostrar el tiempo de entrenamiento
print(f"Tiempo de entrenamiento del modelo: {training_time:.3f} segundos")

# Realizar predicciones con el conjunto de prueba
y_pred = ada_model.predict(x_test)

# Calcular las métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

# Desnormalizar las predicciones y los valores reales
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1))

# Crear un DataFrame para facilitar el trazado
df = pd.DataFrame({'Real': y_test_inv.flatten(), 'Predicho': y_pred_inv.flatten()})

# Trazar los valores reales y predichos usando Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(df)), y=df['Real'], mode='lines', name='Real'))
fig.add_trace(go.Scatter(x=np.arange(len(df)), y=df['Predicho'], mode='lines', name='Predicho'))
fig.update_layout(title={'text': "Comparativa de precios reales y predicciones diarias utilizando ADA Boost", 'x': 0.5},
                  xaxis_title='Observaciones',
                  yaxis_title='Precio',
                  template='plotly_white')
fig.show()

c:\ProgramData\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



Tiempo de entrenamiento del modelo: 4.074 segundos
MAE: 0.01659789357089173
MSE: 0.00047023326118622377
RMSE: 0.021684862489446958
R^2: 0.9579031622272328


![Comparativa de precios reales y predicciones diarias utilizando ADA Boost](Graficas%20de%20resultados/Comparativa%20de%20precios%20reales%20y%20predicciones%20diarias%20utilizando%20ADA%20Boost.png)

In [43]:
from sklearn.model_selection import learning_curve
import plotly.graph_objects as go

def plot_learning_curves(model, X, y):
    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 5))
    
    train_mean = -np.mean(train_scores, axis=1)
    test_mean = -np.mean(test_scores, axis=1)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_sizes, y=train_mean, mode='lines', name='Perdida Entrenamiento'))
    fig.add_trace(go.Scatter(x=train_sizes, y=test_mean, mode='lines', name='Perdida Validación'))
    
    fig.update_layout(title={'text': "Curvas de Aprendizaje de prediccion diaria ADA Boosting", 'x': 0.5},
                      xaxis_title='Tamaño del conjunto de entrenamiento',
                      yaxis_title='Error (MSE)',
                      template='plotly_white')
    fig.show()

# Asegúrate de tener x e y definidos como tus datos de entrada y salida
plot_learning_curves(ada_model, x, y)

![Curvas de Aprendizaje de prediccion diaria ADA Boosting](Graficas%20de%20resultados/Curvas%20de%20Aprendizaje%20de%20prediccion%20diaria%20ADA%20Boosting.png)

### predicciones semanales

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import time
import plotly.graph_objects as go

# Cargar los datos asegurando que la columna 'Date' se analice como fecha
data_brent = pd.read_csv('brent_oil_prices.csv', parse_dates=['Date'])
data_brent.set_index('Date', inplace=True)

# Convertir el índice a DatetimeIndex asegurando que se manejen las zonas horarias
if not isinstance(data_brent.index, pd.DatetimeIndex):
    data_brent.index = pd.to_datetime(data_brent.index, utc=True)

# Agrupar datos por semana y obtener el precio de cierre promedio de la semana
data_brent_weekly = data_brent['Close'].resample('W').mean()

# Manejo de valores NaN - relleno hacia adelante
data_brent_weekly.fillna(method='ffill', inplace=True)

# Preparar los datos para AdaBoost
scaler = MinMaxScaler(feature_range=(0, 1))
data_brent_scaled = scaler.fit_transform(data_brent_weekly.values.reshape(-1, 1))

# Crear secuencias
def create_features(data, seq_length):
    x, y = [], []
    for i in range(seq_length, len(data)):
        x.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(x), np.array(y)

seq_length = 20
x, y = create_features(data_brent_scaled, seq_length)

# División de datos en entrenamiento y prueba
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Instanciar el modelo AdaBoost con DecisionTreeRegressor como base utilizando los mejores hiperparámetros
base_estimator = DecisionTreeRegressor(max_depth=6, min_samples_leaf=4, min_samples_split=10, random_state=42)
ada_model = AdaBoostRegressor(base_estimator=base_estimator, n_estimators=150, learning_rate=0.01, random_state=42)

# Iniciar el cronómetro para calcular el tiempo de entrenamiento
start_time = time.time()

# Entrenar el modelo
ada_model.fit(x_train, y_train)

# Detener el cronómetro
end_time = time.time()
training_time = end_time - start_time

# Mostrar el tiempo de entrenamiento
print(f"Tiempo de entrenamiento del modelo: {training_time:.3f} segundos")

# Realizar predicciones con el conjunto de prueba
y_pred = ada_model.predict(x_test)

# Calcular las métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

# Desnormalizar las predicciones y los valores reales
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1))

# Crear un DataFrame para facilitar el trazado
df = pd.DataFrame({'Real': y_test_inv.flatten(), 'Predicho': y_pred_inv.flatten()})

# Trazar los valores reales y predichos usando Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(df)), y=df['Real'], mode='lines', name='Real'))
fig.add_trace(go.Scatter(x=np.arange(len(df)), y=df['Predicho'], mode='lines', name='Predicho'))
fig.update_layout(title={'text': "Comparativa de precios reales y predicciones semanales utilizando ADA Boost", 'x': 0.5},
                  xaxis_title='Observaciones',
                  yaxis_title='Precio',
                  template='plotly_white')
fig.show()

c:\ProgramData\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Tiempo de entrenamiento del modelo: 0.751 segundos
MAE: 0.024176810755780584
MSE: 0.001053398216466267
RMSE: 0.032456096753403156
R^2: 0.9162276726594379


![Comparativa de precios reales y predicciones semanales utilizando ADA Boost](Graficas%20de%20resultados/Comparativa%20de%20precios%20reales%20y%20predicciones%20semanales%20utilizando%20ADA%20Boost.png)

In [3]:
from sklearn.model_selection import learning_curve
import plotly.graph_objects as go
import numpy as np

def plot_learning_curves(model, X, y):
    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 5))
    
    train_mean = -np.mean(train_scores, axis=1)
    test_mean = -np.mean(test_scores, axis=1)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_sizes, y=train_mean, mode='lines', name='Perdida Entrenamiento'))
    fig.add_trace(go.Scatter(x=train_sizes, y=test_mean, mode='lines', name='Perdidia Validación'))
    
    fig.update_layout(title={'text': "Curvas de Aprendizaje de prediccion semanal ADA Boosting", 'x': 0.5},
                      xaxis_title='Tamaño del conjunto de entrenamiento',
                      yaxis_title='Error (MSE)',
                      template='plotly_white')
    fig.show()

# Asegúrate de tener x e y definidos como tus datos de entrada y salida
plot_learning_curves(ada_model, x, y)

![Curvas de Aprendizaje de prediccion semanal ADA Boosting](Graficas%20de%20resultados/Curvas%20de%20Aprendizaje%20de%20prediccion%20semanal%20ADA%20Boosting.png)

## LSTM

### predicciones diarias

In [46]:
#definitiva diaria TFM

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time  # Importar el módulo time
import plotly.graph_objects as go


# Cargar los datos
data_brent = pd.read_csv('brent_oil_prices.csv')

# Convertir la columna 'Date' a tipo datetime
data_brent['Date'] = pd.to_datetime(data_brent['Date'])

# Establecer la columna 'Date' como índice del DataFrame
data_brent.set_index('Date', inplace=True)

# Normalizar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_brent['Close'].values.reshape(-1, 1))

# Dividir los datos en conjuntos de entrenamiento y prueba
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train_data, test_data = scaled_data[0:train_size], scaled_data[train_size:len(scaled_data)]

# Función para crear secuencias temporales
def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:(i + seq_length), 0])
        y.append(data[i + seq_length, 0])
    return np.array(x), np.array(y)

# Definir la longitud de la secuencia temporal
seq_length = 100

# Crear las secuencias temporales
x_train, y_train = create_sequences(train_data, seq_length)
x_test, y_test = create_sequences(test_data, seq_length)

# Reshape de los datos para que coincidan con el formato de entrada de LSTM [samples, time steps, features]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Definir el modelo LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Iniciar el cronómetro antes de comenzar el entrenamiento
start_time = time.time()

# Entrenar el modelo
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Detener el cronómetro después de completar el entrenamiento
end_time = time.time()

# Calcular la duración del entrenamiento
training_time = end_time - start_time

# Mostrar el tiempo de entrenamiento
print(f"Tiempo de entrenamiento del modelo: {training_time} segundos")

# Realizar predicciones
predictions = model.predict(x_test)

# Invertir la normalización para obtener las predicciones en la escala original
predictions = scaler.inverse_transform(predictions)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calcular las métricas
mae = mean_absolute_error(y_test_inv, predictions)
mse = mean_squared_error(y_test_inv, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_inv, predictions)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

# Graficar los resultados con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_brent.index[-len(predictions):], y=data_brent['Close'][-len(predictions):], mode='lines', name='Precio Real'))
fig.add_trace(go.Scatter(x=data_brent.index[-len(predictions):], y=predictions.flatten(), mode='lines', name='Predicciones'))
fig.update_layout(title= {'text': "Comparativa de precios reales y predicciones diarias utilizando LSTM", 'x': 0.5},
                  xaxis_title='Fecha',
                  yaxis_title='Precio de Cierre (USD)',
                  template='plotly_white')
fig.show()

Epoch 1/10
81/81 [==============================] - 8s 55ms/step - loss: 0.0261 - val_loss: 0.0011
Epoch 2/10
81/81 [==============================] - 4s 46ms/step - loss: 9.3464e-04 - val_loss: 9.3920e-04
Epoch 3/10
81/81 [==============================] - 4s 49ms/step - loss: 8.9491e-04 - val_loss: 8.1574e-04
Epoch 4/10
81/81 [==============================] - 5s 59ms/step - loss: 8.4912e-04 - val_loss: 7.6647e-04
Epoch 5/10
81/81 [==============================] - 5s 58ms/step - loss: 7.8785e-04 - val_loss: 6.9678e-04
Epoch 6/10
81/81 [==============================] - 4s 50ms/step - loss: 7.0955e-04 - val_loss: 6.5546e-04
Epoch 7/10
81/81 [==============================] - 4s 46ms/step - loss: 7.0618e-04 - val_loss: 8.0439e-04
Epoch 8/10
81/81 [==============================] - 4s 54ms/step - loss: 6.6280e-04 - val_loss: 5.9149e-04
Epoch 9/10
81/81 [==============================] - 4s 46ms/step - loss: 6.2678e-04 - val_loss: 5.7272e-04
Epoch 10/10
81/81 [==========================

![Comparativa de precios reales y predicciones diarias utilizando LSTM](Graficas%20de%20resultados/Comparativa%20de%20precios%20reales%20y%20predicciones%20diarias%20utilizando%20LSTM.png)

In [28]:
# Suponiendo que 'history' contiene el historial de entrenamiento, incluyendo la pérdida de entrenamiento y validación
loss = history.history['loss']
val_loss = history.history['val_loss']  # Asegúrate de haber incluido la validación en el entrenamiento para tener esta métrica

# Crear un rango de épocas para el eje x
epochs = range(1, len(loss) + 1)

# Crear la figura de Plotly
fig = go.Figure()

# Agregar la curva de pérdida de entrenamiento
fig.add_trace(go.Scatter(x=list(epochs), y=loss, mode='lines', name='Pérdida de Entrenamiento'))

# Agregar la curva de pérdida de validación
fig.add_trace(go.Scatter(x=list(epochs), y=val_loss, mode='lines', name='Pérdida de Validación'))

# Configurar el layout del gráfico
fig.update_layout(title= {'text': 'Curvas de Aprendizaje de la prediccion diaria del modelo LSTM', 'x': 0.5},
                  xaxis_title='Época',
                  yaxis_title='Pérdida',
                  legend_title='Tipo',
                  template='plotly_white')

# Mostrar el gráfico
fig.show()



![Curvas de Aprendizaje de la prediccion diaria del modelo LSTM](Graficas%20de%20resultados/Curvas%20de%20Aprendizaje%20de%20la%20prediccion%20diaria%20del%20modelo%20LSTM.png)

### predicciones semanales

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.initializers import GlorotUniform  # Bueno para evitar gradientes que explotan/vanish
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math
import time
from keras.callbacks import History


# Cargar los datos asegurando que la columna 'Date' se analice como fecha
data_brent = pd.read_csv('brent_oil_prices.csv', parse_dates=['Date'])
data_brent.set_index('Date', inplace=True)

# Convertir el índice a DatetimeIndex asegurando que se manejen las zonas horarias
if not isinstance(data_brent.index, pd.DatetimeIndex):
    data_brent.index = pd.to_datetime(data_brent.index, utc=True)  # Uso de utc=True para conversiones tz-aware

# Agrupar datos por semana y obtener el precio de cierre promedio de la semana
data_brent_weekly = data_brent['Close'].resample('W').mean()

# Manejo de valores NaN - relleno hacia adelante
data_brent_weekly.interpolate(method='linear', inplace=True)

# Preparar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_brent_weekly.values.reshape(-1, 1))

def create_sequences(data, sequence_length):
    x, y = [], []
    for i in range(len(data) - sequence_length):
        x.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length])
    return np.array(x), np.array(y)

sequence_length = 5
x, y = create_sequences(scaled_data, sequence_length)

# Dividir los datos
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Construir el modelo
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1), kernel_initializer=GlorotUniform(seed=42)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, kernel_initializer=GlorotUniform(seed=42)))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Iniciar el contador de tiempo
start_time = time.time()

# Entrenar el modelo
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Calcular y mostrar el tiempo de entrenamiento
end_time = time.time()
training_time = end_time - start_time
print(f"Tiempo de entrenamiento del modelo: {training_time:.2f} segundos")

# Predicciones
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Ajustar las fechas de predicción
num_test_samples = len(x_test)
adjusted_prediction_dates = data_brent_weekly.index[-num_test_samples-sequence_length+1:-sequence_length+1]
#adjusted_prediction_dates_str = adjusted_prediction_dates.strftime('%Y-%m-%d')

# Ajustar las fechas para las predicciones
#prediction_dates = data_brent_weekly.index[-len(y_test):]
#adjusted_prediction_dates = prediction_dates[sequence_length:]

# Calcular métricas de evaluación
y_test_inv = scaler.inverse_transform(y_test)
mae = mean_absolute_error(y_test_inv, predictions)
mse = mean_squared_error(y_test_inv, predictions)
rmse = math.sqrt(mse)
r2 = r2_score(y_test_inv, predictions)

# Mostrar métricas de evaluación
print(f"MAE (Error Absoluto Medio): {mae}")
print(f"MSE (Error Cuadrático Medio): {mse}")
print(f"RMSE (Raíz del Error Cuadrático Medio): {rmse}")
print(f"R^2 (Coeficiente de Determinación): {r2}")

# Visualización
# Visualizar los resultados con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=adjusted_prediction_dates, y=scaler.inverse_transform(y_test).flatten(), mode='lines', name='precio Real'))
fig.add_trace(go.Scatter(x=adjusted_prediction_dates, y=predictions.flatten(), mode='lines', name='precio Predicción'))
fig.update_layout(title={'text': "Comparativa de precios reales y predicciones semanales utilizando LSTM", 'x': 0.5}, xaxis_title='Fecha', yaxis_title='Precio',
                  template='plotly_white')
fig.show()

Epoch 1/100
18/18 [==============================] - 3s 39ms/step - loss: 0.1372 - val_loss: 0.0228
Epoch 2/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0194 - val_loss: 0.0052
Epoch 3/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0123 - val_loss: 0.0104
Epoch 4/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0074 - val_loss: 0.0048
Epoch 5/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0062 - val_loss: 0.0026
Epoch 6/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0061 - val_loss: 0.0023
Epoch 7/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0056 - val_loss: 0.0024
Epoch 8/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0054 - val_loss: 0.0023
Epoch 9/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0053 - val_loss: 0.0023
Epoch 10/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0057 - val_loss: 0.0024
Epoch 11

![Comparativa de precios reales y predicciones semanales utilizando LSTM](Graficas%20de%20resultados/Comparativa%20de%20precios%20reales%20y%20predicciones%20semanales%20utilizando%20LSTM.png)

In [53]:
import plotly.graph_objects as go

# Crear la figura de Plotly
fig = go.Figure()

# Agregar la curva de pérdida de entrenamiento
fig.add_trace(go.Scatter(
    x=list(range(len(history.history['loss']))),
    y=history.history['loss'],
    mode='lines',
    name='Pérdida de Entrenamiento'
))

# Agregar la curva de pérdida de validación
fig.add_trace(go.Scatter(
    x=list(range(len(history.history['val_loss']))),
    y=history.history['val_loss'],
    mode='lines',
    name='Pérdida de Validación'
))

# Actualizar el layout de la gráfica
fig.update_layout(
    title={
        'text': "Curvas de Aprendizaje de la prediccion semanal del modelo LSTM",
        'x': 0.5
    },
    xaxis_title='Época',
    yaxis_title='Pérdida',
    template='plotly_white'
)

# Mostrar la gráfica
fig.show()


![Curvas de Aprendizaje de la prediccion semanal del modelo LSTM](Graficas%20de%20resultados/Curvas%20de%20Aprendizaje%20de%20la%20prediccion%20semanal%20del%20modelo%20LSTM.png)